# Classification wine

In [2]:
import pandas as pd
import numpy as np

In [3]:
pd.set_option('max_columns', None)

In [9]:
df_wines = pd.read_csv('df_wines_toClassification.csv', usecols=['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar','chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'type'])
df_wines.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,type
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,White
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,White
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,White
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,White
4,6.2,0.32,0.16,7.0,0.045,30.0,136.0,0.9949,3.18,0.47,White


Segun la conclusion del analisis de los datos, para la clasificacion del vino segun su tipo, excluimos el alcohol y quality.

In [10]:
df_wines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5302 entries, 0 to 5301
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         5302 non-null   float64
 1   volatile acidity      5302 non-null   float64
 2   citric acid           5302 non-null   float64
 3   residual sugar        5302 non-null   float64
 4   chlorides             5302 non-null   float64
 5   free sulfur dioxide   5302 non-null   float64
 6   total sulfur dioxide  5302 non-null   float64
 7   density               5302 non-null   float64
 8   pH                    5302 non-null   float64
 9   sulphates             5302 non-null   float64
 10  type                  5302 non-null   object 
dtypes: float64(10), object(1)
memory usage: 455.8+ KB


#### Cambiaremos los valores de type, 0-> Red, 1 -> White

In [11]:
from sklearn.preprocessing import LabelEncoder